In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from skimage import io, transform
import os, glob
import seaborn
from keras import regularizers
from keras.models import Model
from keras.layers import Input, Dense
import matplotlib.pyplot as plt
from skimage import color
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from sklearn.model_selection import train_test_split
print("Loaded...")

Using TensorFlow backend.


Loaded...


In [ ]:
from google.colab import files
files.upload()

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
link = 'https://drive.google.com/open?id=1FXfqwFrDRiPli53KA30AoOs_X-kwF6xY' 
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('modelv2.h5')  


In [ ]:
link = 'https://drive.google.com/open?id=12cOLk2ufTiOyhZLuDxwl8rBBTLLjFlcu' 
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('modelv2.json')  


In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d koryakinp/fingers

In [ ]:
!unzip fingers.zip

In [0]:
!ls 

adc.json     kaggle.json  modelv2.h5	sample_data  train
fingers.zip  __MACOSX	  modelv2.json	test


In [2]:
train_img_list = glob.glob("train/*.png")
test_img_list = glob.glob("test/*.png")
print(len(train_img_list),
     len(test_img_list), sep = '\n')

18000
3600


In [3]:
def import_data():
    train_img_data = []
    test_img_data = []
    train_label_data = []
    test_label_data = []
    
    for img in train_img_list:
        img_read = io.imread(img)
        img_read = transform.resize(img_read, (128,128,1), mode = 'constant')
        #img_read=skimage.color.gray2rgb(img_read)
        train_img_data.append(img_read)
        train_label_data.append(img[-6])
    
    for img in test_img_list:
        img_read = io.imread(img)
        img_read = transform.resize(img_read, (128,128,1), mode = 'constant')
        #img_read=skimage.color.gray2rgb(img_read)
        test_img_data.append(img_read)
        test_label_data.append(img[-6])
        
    return np.array(train_img_data), np.array(test_img_data), np.array(train_label_data), np.array(test_label_data)

In [ ]:
xtrain, xtest, ytrain, ytest = import_data()
#xtrain = xtrain / 255
#xtest = xtest / 255

In [ ]:
xtrain.shape

In [0]:
xtrain = xtrain.reshape(xtrain.shape[0], 128, 128, 1)
xtest = xtest.reshape(xtest.shape[0], 128, 128, 1)

ytrain = tf.keras.utils.to_categorical(ytrain, num_classes = 6)
ytest = tf.keras.utils.to_categorical(ytest, num_classes = 6)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(18000, 128, 128, 3) (3600, 128, 128, 3) (18000, 6) (3600, 6)


In [0]:
x_train, x_test, y_train, y_test = train_test_split(xtrain, ytrain, test_size = 0.20, random_state = 7, shuffle = True)
x_train_val, x_test_val, y_train_val, y_test_val = train_test_split(x_train, y_train, test_size = 0.20, random_state = 7, shuffle = True)

In [0]:
from keras.models import model_from_json
json_file = open('modelv2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("modelv2.h5")
print("Loaded model from disk")






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Loaded model from disk


In [0]:
loaded_model.compile('SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])

#pred = loaded_model.evaluate(xtest,
#                    ytest,
 #                   batch_size = 128)

#print("Accuracy of model on test data is: ",pred[1]*100)

from keras.applications.resnet50 import ResNet50
model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
y = model.output
y = Flatten()(y)
y = Dropout(0.5)(y)
y = Dense(6, activation='softmax', name='class_id')(y)
new_model = Model(inputs=model.input, outputs=y)
new_model.compile('SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])
new_model.fit(x_train,y_train,
              validation_data=(xtest, ytest),
              epochs=20,
             verbose=1)

In [0]:
from skimage import color
import numpy as np
img = io.imread("test10.jpg")
#img_grey = color.rgb2gray(img)
img_predict = transform.resize(img_grey, (128,128,3), mode = 'constant')
plt.imshow(img_predict)
tab = []
tab.append(img_predict)
tab = np.array(tab)
tab = np.expand_dims(tab, axis=3)
tab=loaded_model.predict(tab)
predicted_class = np.argmax(tab[0])
print(predicted_class)

In [0]:
print(xtrain[1].shape)
print(img_predict.shape)

In [0]:
plt.figure(figsize=(16,10))
plt.subplot(2,2,1)
plt.imshow(np.squeeze(xtrain[2]))
plt.subplot(2, 2, 2)
plt.imshow(np.squeeze(xtrain[3]))

In [0]:
plt.imshow(color.rgb2grey(xtrain[1]))